In [64]:
#only support for both sexes for now
code = 'I10'
pheno = '/data/analysis/UKBB/raw/' + code + '.gwas.imputed_v3.both_sexes.tsv'
study = 'cc'
print(pheno)

/data/analysis/UKBB/raw/I10.gwas.imputed_v3.both_sexes.tsv


In [65]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import OrderedDict


In [66]:
dgwas=pd.read_csv(pheno, sep='\t')
phenotype=pd.read_csv('/data/analysis/UKBB/raw/phenotypes.both_sexes.tsv.gz',sep='\t')

In [67]:
print('most significant associations')
print(dgwas.query('~low_confidence_variant').sort_values(['pval'], ascending=[True]).head())

most significant associations
                   variant minor_allele  minor_AF  expected_case_minor_AC  \
4051199    4:182158091:C:T            T  0.018967                 32.8128   
7832246   9:111786116:CA:C            C  0.045338                 78.4349   
11849001   17:42894350:G:A            A  0.020471                 35.4149   
6162668   7:51185016:AAT:A            A  0.073765                127.6130   
11849018   17:42899199:T:G            G  0.020395                 35.2838   

          low_confidence_variant  n_complete_samples       AC       ytx  \
4051199                    False              361194  13701.5   63.0039   
7832246                    False              361194  32751.7  120.7410   
11849001                   False              361194  14788.0   66.1647   
6162668                    False              361194  53287.0  182.9100   
11849018                   False              361194  14733.3   65.4784   

              beta        se    tstat          pval  
40

In [68]:
#Use only high confident variant
dgwas_sub = dgwas.loc[~dgwas['low_confidence_variant'],:]
snp = pd.DataFrame(dgwas_sub.variant.str.split(':',0).tolist(),columns = ['chr','pos', 'ref', 'alt'])
dgwas_sub_snp = pd.concat([snp, dgwas_sub.reset_index()], axis=1)

In [69]:
#use snps in chr 1-22
dgwas_sub_snp_chr = dgwas_sub_snp[dgwas_sub_snp['chr'].isin(list(map(lambda x: str(x), range(1,23))))]

In [70]:
#convert chr/pos type to int
dgwas_sub_snp_chr.chr = dgwas_sub_snp_chr.chr.astype(int)
dgwas_sub_snp_chr.pos = dgwas_sub_snp_chr.pos.astype(int)

/data/env/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [71]:
#use the top 100 associations for fine mapping
dgwas_min = dgwas_sub_snp_chr.nsmallest(100, 'pval')
dgwas_min = dgwas_min.sort_values(['chr', 'pos'], ascending=[True, True])

In [72]:
#select interesting regions
snps = OrderedDict()
for index, row in dgwas_min.iterrows():
    chrs = row['chr']
    pos = row['pos']
    if chrs in snps:
        if any(list(map(lambda x: x[0] <= pos <= x[1], snps[chrs]))):
            continue
        else:
            snps[chrs].append((pos-1000000, pos+1000000))
    else:
        snps[chrs]= [(int(pos)-1000000, int(pos)+1000000)]
print("regions to work with:\n", snps)

regions to work with:
 OrderedDict([(3, [(149461568, 151461568)]), (4, [(98703052, 100703052), (181124566, 183124566)]), (6, [(10491250, 12491250)]), (7, [(10259278, 12259278), (50152099, 52152099)]), (8, [(35056701, 37056701)]), (9, [(89790312, 91790312), (110786116, 112786116)]), (10, [(71684850, 73684850)]), (11, [(108704645, 110704645), (115037160, 117037160)]), (12, [(81547495, 83547495)]), (15, [(36229967, 38229967)]), (17, [(41728733, 43728733), (68392323, 70392323)])])


In [73]:
#get snps in the interesting regions
n = 1
for chrs in snps: 
    print("processing chr",chrs)
    for region in snps[chrs]:
        if n == 1:
            table = dgwas_sub_snp_chr.loc[(dgwas_sub_snp_chr["chr"] == chrs) & ( dgwas_sub_snp_chr["pos"] >= region[0]) & 
       (dgwas_sub_snp_chr["pos"] <= region[1]), ]
            table['SEGNUMBER'] = n
        else:
            temp = dgwas_sub_snp_chr.loc[(dgwas_sub_snp_chr["chr"] == chrs) & ( dgwas_sub_snp_chr["pos"] >= region[0]) & 
       (dgwas_sub_snp_chr["pos"] <= region[1]), ]
            temp['SEGNUMBER'] = n
            table = table.append(temp)
    n = n + 1

processing chr 3
processing chr 4
processing chr 6


/data/env/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/data/env/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


processing chr 7
processing chr 8
processing chr 9
processing chr 10
processing chr 11
processing chr 12
processing chr 15
processing chr 17


In [74]:
table['snp'] = table["chr"].map(str) + "_" + table["pos"].map(str)

#if SNPs have the same position, choose the SNP with the smallest pvalue
indices = table.groupby('snp')['pval'].idxmin
table_unique = table.loc[indices]

In [75]:
table_unique = table_unique.sort_values(['chr', 'pos'], ascending=[True, True])

In [76]:
res = table_unique[['variant', 'chr', 'pos', 'tstat', 'minor_AF', 'SEGNUMBER', 'pval']]

res = res.rename(columns={'variant': 'SNPID', 'chr': 'CHR', 'pos': 'POS',
                   'tstat': 'Z', 'minor_AF': 'F'})

In [77]:
if study == 'cc':
    res['NCONTROL'] = int(phenotype.loc[phenotype['phenotype']==code]['n_controls'])
    res['NCASE'] = int(phenotype.loc[phenotype['phenotype']==code]['n_cases'])
else:
    res['N'] = int(phenotype.loc[phenotype['phenotype']==code]['n_non_missing'])

In [78]:
res.to_csv('/data/analysis/UKBB/processed/'+ code +'.gwas.imputed_v3.both_sexes.finemapping.tsv.gz',
           sep=' ',header=True, index=False, compression="gzip")